In [1]:
import numpy as np
from kaggle_environments import make, evaluate
from gym import spaces

Loading environment football failed: No module named 'gfootball'


In [2]:
class ConnectFourGym:
    def __init__(self, agent2='random'):
        ks_env = make('connectx', debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        
        # Learn about spaces: https://gym.openai.com/docs/#spaces
        self.action_space = spaces.Discrete(self.columns)
        self.observation_space = spaces.Box(low=0, high=2,
                                           shape=(self.rows, self.columns, 1),
                                           dtype=np.int)
        
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
    
    def reset(self):
        self.obs = self.env.reset()
        return np.array(self.obs['board']).reshape(self.rows, self.columns, 1)
    
    def change_reward(self, old_reward, done):
        if old_reward == 1: # Agent won
            return 1
        elif done: # opponent won
            return -1
        else: # Reward 1/42
            return 1 / (self.rows * self.columns)
        
    def step(self, action):
        # Check if agent's move is valid
        is_valid = (self.obs['board'][int(action)] == 0)
        
        if is_valid: # play the move
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -10, True, {}
        
        return np.array(self.obs['board']).reshape(self.rows, self.columns, 1), reward, done, _

In [3]:
env = ConnectFourGym(agent2='random')

In [4]:
import os
from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv

# Create directory for logging training information
log_dir = 'ppo/'
os.makedirs(log_dir, exist_ok=True)

# Logging progress
monitor_env = Monitor(env, log_dir, allow_early_resets=True)

# Create a vectorized environment
vec_env = DummyVecEnv([lambda: monitor_env])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



ImportError: DLL load failed: The specified module could not be found.

In [ ]:
from stable_baselines import PP01
from stable_baselines.a2c.utils import conv, linear, conv_to_fc
from stable_baselines.common.policies import CnnPolicy

# Neural network for predicting action values
def modified_cnn(scaled_images, **kwargs):
    activ = tf.nn.relu
    layer_1 = activ(conv(scaled_images, 'c1', n_filters=32, filter_size=3, stride=1, 
                         init_scale=np.sqrt(2), **kwargs))
    layer_2 = activ(conv(layer_1, 'c2', n_filters=64, filter_size=3, stride=1,
                         init_scale=np.sqrt(2), **kwargs))
    layer_2 = conv_to_fc(layer_2)
    return activ(linear(layer_2, 'fc1', n_hidden=512, init_scale=np.sqrt(2)))

class CustomCNNPolicy(CnnPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomCNNPolicy, self).__init__(*args, **kwargs, cnn_extractor=modified_cnn)
        
# Initialize agent
model = PP01(CustomCNNPolicy, vec_env, verbose=0)

Fed up with errors with MPI. I'm moving on and using the Kaggle notebook for the rest of this lesson